<a href="https://colab.research.google.com/github/skandanyal/ML_Lab_2025_2026/blob/main/lab_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Implementing Bayesian Decision Theory
**Objective:** Implement Bayesian Decision Theory for classification.

In [1]:



import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report


iris = load_iris()
X = iris.data
y = iris.target
class_labels = iris.target_names


X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)


scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


n_classes = len(np.unique(y_train))
n_features = X_train.shape[1]


priors = np.zeros(n_classes)
for c in range(n_classes):
    priors[c] = np.mean(y_train == c)


means = np.zeros((n_classes, n_features))
covariances = []

for c in range(n_classes):
    X_c = X_train[y_train == c]
    means[c] = np.mean(X_c, axis=0)
    covariances.append(np.cov(X_c, rowvar=False))


def multivariate_gaussian(x, mean, cov):
    size = len(x)
    det = np.linalg.det(cov)
    norm_const = 1.0 / (np.power((2 * np.pi), float(size) / 2) * np.sqrt(det))
    x_mu = x - mean
    inv = np.linalg.inv(cov)
    result = np.exp(-0.5 * (x_mu @ inv @ x_mu.T))
    return norm_const * result


def predict_bdt(X):
    y_pred = []
    for x in X:
        posteriors = []
        for c in range(n_classes):
            likelihood = multivariate_gaussian(x, means[c], covariances[c])
            posterior = priors[c] * likelihood
            posteriors.append(posterior)
        y_pred.append(np.argmax(posteriors))
    return np.array(y_pred)


y_pred = predict_bdt(X_test)


print("=== Bayesian Decision Theory Classification ===\n")
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nAccuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred, target_names=class_labels))

=== Bayesian Decision Theory Classification ===

Confusion Matrix:
 [[15  0  0]
 [ 0 15  0]
 [ 0  1 14]]

Accuracy: 0.9777777777777777

Classification Report:
               precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        15
  versicolor       0.94      1.00      0.97        15
   virginica       1.00      0.93      0.97        15

    accuracy                           0.98        45
   macro avg       0.98      0.98      0.98        45
weighted avg       0.98      0.98      0.98        45

